## HLW_Covid

In [ ]:
from bok_da.ts.nir import HLW_Covid

In [2]:
# !pip install openpyxl
import pandas as pd
import numpy as np
import datetime

In [3]:
KORData = pd.read_excel("data/nir_data_all.xlsx", sheet_name="raw_data")
end_date = datetime.datetime.strptime('2024-07-01', '%Y-%m-%d')
KORData_before = KORData[KORData['Date'] <= end_date]
data = KORData_before.set_index("Date")
data_list = ['rgdp', 'inf1', 'interest', 'csi', 'ci']
data = data[data_list]
data['inf1'] = data['inf1'].pct_change(4) * 100

if smoothing:
    data['inf1'] = data['inf1'].rolling(window=4).mean()

data['rgdp'] = np.log(data['rgdp']) * 400
data['interest'] = data['interest'] - data['inf1']
data = data.dropna()
#data = data.loc['1991-01-01':]
KORmacro = data['2000-01-01':].copy()
#KORmacro = data.copy()
def shift_quarter_end(date):
    if date.month == 1:
        return date.replace(month=3, day=31)
    elif date.month == 4:
        return date.replace(month=6, day=30)
    elif date.month == 7:
        return date.replace(month=9, day=30)
    else:
        return date.replace(month=12, day=31)

KORmacro.index = KORmacro.index.map(shift_quarter_end)
KORmacro

,rgdp,inf1,interest,csi,ci
Date,,,,,
2000-03-31,4961.268953,1.227883,3.722117,0.0,0
2000-06-30,4967.384872,1.573023,3.526977,0.0,0
2000-09-30,4978.220343,2.196806,2.923194,0.0,0
2000-12-31,4976.889295,2.479182,2.850818,0.0,0
2001-03-31,4980.892991,3.258236,1.881764,0.0,0
...,...,...,...,...,...
2023-09-30,5296.439326,3.742613,-0.194613,0.0,0
2023-12-31,5298.255260,3.294664,0.281336,0.0,0
2024-03-31,5303.422191,2.566453,0.939547,0.0,0


In [ ]:
mcmc =  {
            'sig2_g': 0,
            'sig2_yt': 0,
            'sig2_yc': 0,
            'sig2_z': 0,
            'sig2_p': 0,
            'slope_yc': 0,
            'slope_z': 0,
            'slope_p': 0
        }

if smoothing:
    prior = {
        'yc_mean': np.array([0.7, 0.2, -0.2]), # [0.7, 0.2, -0.3]
        'yc_variance': np.diag(np.array([0.1, 0.1, 0.03]) ** 2), # np.diag(np.array([0.1, 0.1, 0.03]) ** 2)
        'sig2_yc_shape': 5,
        'sig2_yc_scale': 4 * 2**2,
        'sig2_g_shape': 5,
        'sig2_g_scale': 4 * 0.1**2,
        'sig2_yt_shape': 5,
        'sig2_yt_scale': 4 * 2**2,
        'sig2_z_shape': 5,
        'sig2_z_scale': 4 * 0.5**2,
        'p_mean': np.array([0.7, 0.1]),
        'p_variance': np.diag(np.array([0.1, 0.1]) ** 2),
        'sig2_p_shape': 5,
        'sig2_p_scale': 4 * 0.1**2,
        'yt_mean': -0.5, # 0
        'yt_variance': 0.1**2, # 10**2
        'kappa20_shape': 3, #5,
        'kappa20_scale': 2, #4*(10**2),
        'kappa21_shape': 3, #5,
        'kappa21_scale': 2, #4*(10**2),
        'kappa22_shape': 3, #5,
        'kappa22_scale': 2 #4*(10**2)
    }
else:
    prior = {
        'yc_mean': np.array([0.7, 0.2, -0.3]),
        'yc_variance': np.diag(np.array([0.1, 0.1, 0.03]) ** 2),
        'sig2_yc_shape': 5,
        'sig2_yc_scale': 4 * 2**2,
        'sig2_g_shape': 5,
        'sig2_g_scale': 4 * 0.1**2,
        'sig2_yt_shape': 5,
        'sig2_yt_scale': 4 * 2**2,
        'sig2_z_shape': 5,
        'sig2_z_scale': 4 * 0.5**2,
        'p_mean': np.array([0.7, 0.1]),
        'p_variance': np.diag(np.array([0.1, 0.1]) ** 2),
        'sig2_p_shape': 5,
        'sig2_p_scale': 4 * 0.1**2,
        'yt_mean': -0.5, # 0
        'yt_variance': 0.1**2, # 10**2
        'kappa20_shape': 3, #5,
        'kappa20_scale': 2, #4*(10**2),
        'kappa21_shape': 3, #5,
        'kappa21_scale': 2, #4*(10**2),
        'kappa22_shape': 3, #5,
        'kappa22_scale': 2, #4*(10**2)
    }

model = HLW_Covid(n_draws=2500, burnin=500, MCMCAlgorithm=mcmc, PriorPara=prior)

> Initializing Model...
  - Total MCMC iterations: 2500
  - Burn-in period: 500
  - Latent variables percentiles: [5, 95]
  - Parameters percentiles: [0.05, 0.95]
  - Interest rate z-component specification: rw


In [14]:
model.fit(KORmacro.loc[:end_period], dates=KORmacro.loc[:end_period].index)

> Starting model fitting...
  - Data shape: (135, 5)
  - Dates provided with length: 135


Step 3/3 :  63%|██████▎   | 1566/2500 [02:45<01:23, 11.24it/s]

In [12]:
rstar_HLW_Covid = pd.DataFrame(model.SummaryLV['Avg']['rNatural'])
rstar_HLW_Covid.index = KORmacro.loc[KORmacro.index[4]:end_period].index
if smoothing:
    rstar_HLW_Covid.columns = ['rstar_HLW_Covid_smoothed']
else:
    rstar_HLW_Covid.columns = ['rstar_HLW_Covid']
rstar = rstar_HLW_Covid.reset_index().rename(columns={'index': 'Date'}).to_json(orient='records', date_format='iso')
rstar_HLW_Covid.reset_index().rename(columns={'index': 'Date'}) \
    .assign(Date=lambda df: df['Date'].dt.strftime('%Y-%m-%d')) \
    .to_json('rstar_HLW_Covid.json', orient='records')

,rstar_HLW
Date,
2004-01-01,1.991575
2004-04-01,1.901086
2004-07-01,1.815172
2004-10-01,1.756367
2005-01-01,1.747494
...,...
2023-07-01,-2.059404
2023-10-01,-2.048616
2024-01-01,-2.046529
